# Hi!ckaton
This notebook presents the work the group n°12 has been producing.

In [1]:
# necessary imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

In [2]:
# load the training dataset
df_train =  pd.read_csv("hfactory_magic_folders/water_shortage_prediction/X_test_Hi5.csv")

/tmp/ipykernel_21997/1538058322.py:2: DtypeWarning: Columns (1,5,108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train =  pd.read_csv("hfactory_magic_folders/water_shortage_prediction/X_test_Hi5.csv")


In [3]:
# first exploration of the dataset
rows, columns = df_train.shape

print(f"Number of rows : {rows}")
print(f"Number of columns : {columns}")

Number of rows : 611208
Number of columns : 135


In [ ]:
usefull_columns = [ 'piezo_station_altitude',
                   'piezo_station_longitude',
                   'piezo_station_latitude',
                   'piezo_station_bss_id',
                   'piezo_station_investigation_depth',
                   'piezo_measurement_date', # To process
                   'piezo_qualification', # To process (discrimination)
                   'meteo_rain_height', # High impact
                   'meteo_temperature_avg',
                   'hydro_observation_result_elab',
                   'hydro_hydro_quantity_elab'] # To process, according to its units
                   


df_train = df_train[usefull_columns]
df_train.shape

(611208, 10)

In [ ]:
df_train['piezo_measurement_date'] = pd.to_datetime(df_train['piezo_measurement_date'])
curr_id = df_train['piezo_station_bss_id'][0]
past_measure = [(df_train['piezo_measurement_date'][0],df_train['meteo_rain_height'][0])]

for i in range(1,df_train.shape[0]):
    if df_train['piezo_station_bss_id'][i] == curr_id:
        for j in range(max(0,len(past_measure) - 10), len(past_measure)):

            stmp = (df_train['piezo_measurement_date'][i] - past_measure[j][0]).days
            if 0 < stmp < 10:
                df_train[f'piezo_measurement_day_-{stmp}'][i] = past_measure[j][1]
        past_measure.append((df_train['piezo_measurement_date'][i],df_train['meteo_rain_height'][i]))
                
    else:
        curr_id = df_train['piezo_station_bss_id'][i]
        past_measure = [(df_train['piezo_measurement_date'][i],df_train['meteo_rain_height'][i])]
    if i%100000 == 0:
        print(i)

(611208, 10)

In [ ]:
df_train = df_train.drop(columns = ['piezo_station_bss_id'])

In [ ]:
# The dates (drop the day information and create a month and a year column)
df_train['piezo_measurement_date'] = pd.to_datetime(df_train['piezo_measurement_date'])

df_train['day'] = df_train['piezo_measurement_date'].dt.day 
df_train['month'] = df_train['piezo_measurement_date'].dt.month
df_train['year'] = df_train['piezo_measurement_date'].dt.year
df_train = df_train.drop(columns = ['piezo_measurement_date']) 

/tmp/ipykernel_21997/3213209030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['piezo_measurement_date'] = pd.to_datetime(df_train['piezo_measurement_date'])
/tmp/ipykernel_21997/3213209030.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['day'] = df_train['piezo_measurement_date'].dt.day
/tmp/ipykernel_21997/3213209030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [9]:
df_train.shape

(611208, 12)

In [10]:
df_train['hydro_observation_result_elab'].describe()

count    6.112080e+05
mean     1.443601e+04
std      7.231818e+04
min     -1.369000e+04
25%      1.020000e+02
50%      4.880000e+02
75%      2.884000e+03
max      1.738213e+06
Name: hydro_observation_result_elab, dtype: float64

In [11]:
# Adapt the value of hydro_hydro_quantity_elab with the right unit
df_train['hydro_observation_result_elab'] = df_train['hydro_observation_result_elab'] / 30
df_train = df_train.drop(columns = ['hydro_hydro_quantity_elab']) 

In [12]:
df_train['hydro_observation_result_elab']

0             6.366667
1           444.866667
2           538.933333
3         11119.666667
4             0.433333
              ...     
611203       12.833333
611204       12.833333
611205       34.000000
611206       10.633333
611207        3.766667
Name: hydro_observation_result_elab, Length: 611208, dtype: float64

In [13]:
df_train.columns

Index(['piezo_station_altitude', 'piezo_station_longitude',
       'piezo_station_latitude', 'piezo_station_investigation_depth',
       'piezo_qualification', 'meteo_rain_height', 'meteo_temperature_avg',
       'hydro_observation_result_elab', 'day', 'month', 'year'],
      dtype='object')

In [14]:
df_train.shape

(611208, 11)

In [16]:
pd.set_option('display.max_columns', None)
df_train.head(10)

,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_station_investigation_depth,piezo_qualification,meteo_rain_height,meteo_temperature_avg,hydro_observation_result_elab,day,month,year
0,232.00,5.748241,45.828148,20.00,Correcte,7.4,17.1,6.366667,1,6,2022
1,247.25,5.356637,46.028102,35.60,Correcte,3.4,16.8,444.866667,1,6,2022
2,218.77,5.220795,45.895734,35.22,Correcte,NaN,18.0,538.933333,1,6,2022
3,499.85,5.948977,46.201180,34.20,Correcte,12.8,11.9,11119.666667,1,6,2022
4,260.00,5.313353,46.136402,37.30,Correcte,2.2,16.4,0.433333,1,6,2022
5,250.00,5.679690,45.874738,50.00,Correcte,5.4,14.4,6.366667,1,6,2022
6,208.21,5.285156,45.837397,21.00,Correcte,3.4,16.8,11394.966667,1,6,2022
7,558.68,6.069710,46.329030,78.00,Correcte,14.8,14.6,0.600000,1,6,2022
8,250.45,5.823274,45.919630,10.00,Correcte,2.6,15.3,10835.200000,1,6,2022
9,265.22,4.830754,46.018341,62.60,Correcte,1.6,18.7,3.433333,1,6,2022


In [29]:
df_train.to_csv('X_test.csv')